In [1]:
from bids.grabbids import BIDSLayout

In [2]:
layout = BIDSLayout("/data/ds102/")

In [3]:
layout.get_subjects()

['01', '02', '03', '04', '05']

In [4]:
layout.get_modalities()

['anat', 'func']

In [5]:
layout.get_types()

['bold', 'events', 'T1w']

In [6]:
layout.get_tasks()

['flanker']

In [7]:
layout.get(subject='01')

[File(filename='/data/ds102/sub-01/anat/sub-01_T1w.nii.gz', type='T1w', modality='anat', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', type='bold', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_events.tsv', type='events', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz', type='bold', task='flanker', run='2', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_events.tsv', type='events', task='flanker', run='2', modality='func', subject='01')]

In [8]:
layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])

[File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', type='bold', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz', type='bold', task='flanker', run='2', modality='func', subject='01')]

In [9]:
[f.filename for f in layout.get(subject='01', type='T1w', extensions=['nii', 'nii.gz'])]

['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']

In [10]:
def get_niftis(subject_id, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    bolds = [f.filename for f in layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])]
    T1ws = [f.filename for f in layout.get(subject='01', type='T1w', extensions=['nii', 'nii.gz'])]
    return bolds, T1ws

In [11]:
get_niftis('01', '/data/ds102')

(['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz',
  '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz'],
 ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz'])

In [12]:
from nipype.pipeline import Node, Workflow
from nipype.interfaces.utility import IdentityInterface, Function

In [13]:
BIDSDataGrabber = Node(Function(function=get_niftis, input_names=["subject_id",
                                       "data_dir"],
                                   output_names=["bolds", 
                                        "T1ws"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.data_dir = "/data/ds102"

In [14]:
BIDSDataGrabber.inputs.subject_id='01'
res = BIDSDataGrabber.run()
res.outputs

170324-02:14:41,668 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpeiZkoI/BIDSDataGrabber



T1ws = ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']
bolds = ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']

In [15]:
def printMe(paths):
    print("analyzing " + str(paths))
    
analyzeBOLD = Node(Function(function=printMe, input_names=["paths"],
                                   output_names=[]), name="analyzeBOLD")

analyzeT1w = Node(Function(function=printMe, input_names=["paths"],
                                   output_names=[]), name="analyzeT1w")


In [16]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD, "paths")
wf.connect(BIDSDataGrabber, "T1ws", analyzeT1w, "paths")
wf.run()

170324-02:14:41,842 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-02:14:41,997 workflow INFO:
	 Running serially.
170324-02:14:41,998 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpSRZQnw/bids_demo/BIDSDataGrabber
170324-02:14:42,81 workflow INFO:
	 Executing node analyzeBOLD in dir: /tmp/tmphxmOJc/bids_demo/analyzeBOLD
analyzing ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']
170324-02:14:42,121 workflow INFO:
	 Executing node analyzeT1w in dir: /tmp/tmpx5Wgao/bids_demo/analyzeT1w
analyzing ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']


In [17]:
BIDSDataGrabber.iterables = ('subject_id', layout.get_subjects())
wf.run()

170324-02:14:42,186 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-02:14:42,307 workflow INFO:
	 Running serially.
170324-02:14:42,309 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a4 in dir: /tmp/tmp9T8lxs/bids_demo/_subject_id_05/BIDSDataGrabber
170324-02:14:42,361 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmpnML3R5/bids_demo/_subject_id_01/BIDSDataGrabber
170324-02:14:42,420 workflow INFO:
	 Executing node analyzeBOLD.a4 in dir: /tmp/tmpFsm8Gt/bids_demo/_subject_id_05/analyzeBOLD
analyzing ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']
170324-02:14:42,453 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmpfEVhfN/bids_demo/_subject_id_02/BIDSDataGrabber
170324-02:14:42,515 workflow INFO:
	 Executing node analyzeT1w.a1 in dir: /tmp/tmpAUChZ_/bids_demo/_subject_id_02/analyzeT1w
analyzing ['/data/ds102/sub-01/anat/su